In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import tensorflow as tf
from tensorflow import keras

In [2]:
data=pd.read_csv('TATAMOTORS.NS.csv',date_parser=True)
data.tail()


,Date,Open,High,Low,Close,Adj Close,Volume
6477,2021-09-07,297.350006,297.799988,290.899994,294.600006,294.600006,14325904.0
6478,2021-09-08,293.500000,298.799988,291.600006,295.250000,295.250000,14920479.0
6479,2021-09-09,294.000000,303.500000,293.549988,298.950012,298.950012,34020970.0
6480,2021-09-13,299.000000,302.000000,295.500000,301.200012,301.200012,16339860.0
6481,2021-09-14,302.250000,307.200012,302.200012,306.100006,306.100006,23234649.0


In [3]:
trdata=data[data['Date']<'2021-01-01'].copy()
tsdata=data[data['Date']>='2021-01-01'].copy()

In [4]:


trdata=trdata.drop(['Date','Adj Close'],axis=1)
trdata.dropna(axis=0,how='any',inplace=True)
print(trdata.isnull().sum())



Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64


In [5]:
s=MinMaxScaler()
trdata=s.fit_transform(trdata)
trdata

array([[0.10474505, 0.10296286, 0.10674933, 0.10488202, 0.        ],
       [0.10474505, 0.10296286, 0.10674933, 0.10488202, 0.        ],
       [0.10474505, 0.10296286, 0.10674933, 0.10488202, 0.        ],
       ...,
       [0.30009534, 0.29753862, 0.29381225, 0.2933923 , 0.33097816],
       [0.2919473 , 0.29240626, 0.29260292, 0.29458507, 0.17902705],
       [0.29491795, 0.29594002, 0.29674913, 0.2940739 , 0.22730211]])

In [6]:
Xtrain=[]
Ytrain=[]

for i in range(90,trdata.shape[0]):
    Xtrain.append(trdata[i-90:i])
    Ytrain.append(trdata[i,0])

In [7]:
Xtrain = np.array(Xtrain)
Ytrain = np.array(Ytrain)

In [8]:
Xtrain.shape,Ytrain.shape


((6205, 90, 5), (6205,))

In [9]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM , Dropout
from keras import optimizers
optimizer = tf.keras.optimizers.Adam(clipvalue=0.5)
from sklearn.metrics import mean_squared_error

In [10]:
reg=Sequential()
reg.add(LSTM(units=60,activation='relu',return_sequences= True, input_shape=(Xtrain.shape[1],5)))
reg.add(Dropout(0.2))
reg.add(LSTM(units=80,activation='relu',return_sequences= True))
reg.add(Dropout(0.2))
reg.add(LSTM(units=100,activation='relu',return_sequences= True))
reg.add(Dropout(0.2))
reg.add(LSTM(units=120,activation='relu'))
reg.add(Dropout(0.2))
reg.add(Dense(units=1))

In [11]:
reg.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 90, 60)            15840     
_________________________________________________________________
dropout (Dropout)            (None, 90, 60)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 90, 80)            45120     
_________________________________________________________________
dropout_1 (Dropout)          (None, 90, 80)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 90, 100)           72400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 90, 100)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 120)               1

In [12]:
reg.compile(optimizer=optimizer, loss='mean_squared_error',metrics=['mse'])

In [ ]:
reg.fit(Xtrain,Ytrain,epochs=10,batch_size=36)

Epoch 1/10
173/173 [==============================] - 28s 148ms/step - loss: 0.0095 - mse: 0.0095
Epoch 2/10
173/173 [==============================] - 27s 159ms/step - loss: 0.0020 - mse: 0.0020
Epoch 3/10
173/173 [==============================] - 28s 161ms/step - loss: 0.0018 - mse: 0.0018
Epoch 4/10
173/173 [==============================] - 28s 163ms/step - loss: 0.0015 - mse: 0.0015
Epoch 5/10
173/173 [==============================] - 29s 166ms/step - loss: 0.0017 - mse: 0.0017
Epoch 6/10
173/173 [==============================] - 29s 167ms/step - loss: 0.0012 - mse: 0.0012
Epoch 7/10
173/173 [==============================] - 29s 169ms/step - loss: 0.0012 - mse: 0.0012
Epoch 8/10
173/173 [==============================] - 30s 171ms/step - loss: 0.0012 - mse: 0.0012
Epoch 9/10
173/173 [==============================] - 30s 172ms/step - loss: 0.0010 - mse: 0.0010
Epoch 10/10
158/173 [==========================>...] - ETA: 2s - loss: 0.0012 - mse: 0.0012

In [ ]:
tsdata.head()

In [ ]:
t=tsdata.tail(90)

In [ ]:
ts=t.append(tsdata,ignore_index=True)
ts=ts.drop(['Adj Close','Date'],axis=1)
ts

In [ ]:
inp=s.transform(ts)

In [ ]:
Xtest=[]
Ytest=[]
for i in range(90,inp.shape[0]):
    Xtest.append(inp[i-90:i])
    Ytest.append(inp[i,0])

In [ ]:
Xtest = np.array(Xtest)
Ytest = np.array(Ytest)
Xtest.shape

In [ ]:
ypred=reg.predict(Xtest)

In [ ]:
ypred

In [ ]:
s.scale_

In [ ]:
scale=1/1.69751181e-03
scale

In [ ]:
ypred=ypred*scale
Ytest=Ytest*scale

In [ ]:

#Visualization 
plt.figure(figsize=(14,5))
plt.plot(Ytest,color ='red',label="Real stock Price")
plt.plot(ypred,color='blue',label="Predicted stock Price")
plt.xlabel('Time')
plt.ylabel('P')